In [1]:
!pip -q install pandas
!pip -q install numpy
!pip -q install transformers
!pip -q install torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.5 MB/s eta 0:00:00


In [4]:
import pandas as pd

# Load the dataset
df = pd.read_csv("/content/train.csv" , nrows=5000)

# Print the first few rows of the dataset
df.head()

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [5]:
df.shape

(5000, 3)

In [6]:
df.isna().any()
df.dtypes

Class Index     int64
Title          object
Description    object
dtype: object

In [7]:
df = df.drop('Title', axis=1)

In [8]:
from transformers import BertModel, BertTokenizer
import torch

In [9]:
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
def extract_features(text):
    # Tokenize the text
    input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    # Get the hidden states for each token
    with torch.no_grad():
        outputs = model(input_ids)
        hidden_states = outputs[2]
    # Concatenate the last 4 hidden states
    token_vecs = []
    for layer in range(-4, 0):
        token_vecs.append(hidden_states[layer][0])
    # Calculate the mean of the last 4 hidden states
    features = []
    for token in token_vecs:
        features.append(torch.mean(token, dim=0))
    # Return the features as a tensor
    return torch.stack(features)

In [11]:
features = []
for i in range(len(df)):
    features.append(extract_features(df.iloc[i]["Description"]))
# Concatenate the features and convert to a numpy array
features = torch.cat(features).numpy()

In [12]:
features

array([[ 0.19557545, -0.3094374 ,  0.34961635, ..., -0.25235352,
         0.7551447 , -0.26609316],
       [ 0.22537161, -0.63355994,  0.475045  , ..., -0.24909757,
         0.5154944 , -0.51390064],
       [ 0.24892975, -0.4213285 ,  0.35271513, ..., -0.32380027,
         0.4754412 , -0.3927046 ],
       ...,
       [-0.18318173, -0.36782834,  0.5075258 , ..., -0.23772165,
         0.5479422 , -0.00453943],
       [-0.29183567, -0.3660308 ,  0.5755134 , ..., -0.2159129 ,
         0.36525732, -0.01131801],
       [-0.16464767, -0.2638927 ,  0.29889432, ...,  0.01551178,
         0.29276526,  0.02550351]], dtype=float32)

In [13]:
import numpy as np
np.save("ag_news_features.npy", features)

In [15]:
labels = df['Class Index'].values
labels

array([3, 3, 3, ..., 1, 3, 3])

In [16]:
print(features.shape)
print(labels.shape)

(20000, 768)
(5000,)


In [18]:
# labels is a 1D numpy array of size 1000
# reshape the feature array to size 1000x(768*4)

features_reshaped = features.reshape((5000, -1))

# concatenate the feature array with the label array horizontally
dataset = np.hstack((features_reshaped, labels.reshape((-1, 1))))

In [ ]:
dataset.shape

In [19]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Convert the training and testing sets back into separate feature and label arrays
X_train, y_train = train_data[:, :-1], train_data[:, -1]
X_test, y_test = test_data[:, :-1], test_data[:, -1]

In [20]:
from sklearn.linear_model import LogisticRegression

# Train a logistic regression classifier on the training set
clf = LogisticRegression(max_iter = 1000)
clf.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [21]:
score = clf.score(X_test, y_test)
print("Accuracy:", score)

Accuracy: 0.9


In [22]:
y_pred = clf.predict(X_test)

In [ ]:
for value in y_pred:
    if value == 0 or value == 1 or value == 3:
        print("Not related to sports")
    elif value == 2:
        print("Sports related")

In [24]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

In [30]:
import pickle


model_path = '/content/model/model.pkl'

# Save the model using pickle
with open(model_path, 'wb') as file:
    pickle.dump(clf, file)

# Print a success message
print("Model saved successfully!")

Model saved successfully!


In [ ]:
import pickle

# Path to the saved model
model_path = '/content/model/model.pkl'

# Load the model from the pickle file
with open(model_path, 'rb') as file:
    clf = pickle.load(file)

In [ ]:
from transformers import BertModel, BertTokenizer
import torch

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def extract_features(text):
    # Tokenize the text
    input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    # Get the hidden states for each token
    with torch.no_grad():
        outputs = model(input_ids)
        hidden_states = outputs[2]
    # Concatenate the last 4 hidden states
    token_vecs = []
    for layer in range(-4, 0):
        token_vecs.append(hidden_states[layer][0])
    # Calculate the mean of the last 4 hidden states
    features = []
    for token in token_vecs:
        features.append(torch.mean(token, dim=0))
    # Return the features as a tensor
    return torch.stack(features)

In [31]:
text="AP - Tiger Woods came up empty in a major again, but he's still the No. 1 ranked player in the world."

In [36]:
features=extract_features(text)

In [37]:
features = features.numpy()

In [39]:
features.shape

(4, 768)

In [40]:
features_reshaped = features.reshape((1, -1))

In [43]:
y_pred = clf.predict(features_reshaped)

In [44]:
y_pred

array([2.])

In [45]:
for value in y_pred:
    if value == 0 or value == 1 or value == 3:
        print("Not related to sports")
    elif value == 2:
        print("Sports related")

Sports related
